In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file(
    "aclImdb_v1", url,
    untar=True, cache_dir='.',
    cache_subdir=''
)

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')


84125825/84125825 [==============================] - 260s 3us/step


In [3]:
os.listdir(dataset_dir)

['imdb.vocab', 'imdbEr.txt', 'README', 'test', 'train']

In [4]:
train_dir = os.path.join(dataset_dir,"train")
os.listdir(train_dir)

['labeledBow.feat',
 'neg',
 'pos',
 'unsup',
 'unsupBow.feat',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt']

In [5]:
shutil.rmtree(os.path.join(train_dir,"unsup"))

In [6]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [7]:
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [8]:
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size=batch_size)

Found 25000 files belonging to 2 classes.


In [9]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '', ' ')
    return tf.strings.regex_replace(
        stripped_html,
        f'[{re.escape(string.punctuation)}]',
        ''
    )

In [10]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [11]:
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [12]:
def vectorize_text(text,label):
    text = tf.expand_dims(text,-1)
    return vectorize_layer(text),label

In [13]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = raw_train_ds.map(vectorize_text).cache().prefetch(buffer_size=AUTOTUNE)
val_ds = raw_val_ds.map(vectorize_text).cache().prefetch(buffer_size=AUTOTUNE)
test_ds = raw_test_ds.map(vectorize_text).cache().prefetch(buffer_size=AUTOTUNE)  

In [14]:
embedding_dim = 16

model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Conv1D(8,7,activation="relu"),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(8,activation="relu"),
  layers.Dense(1)])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160016    
                                                                 
 conv1d (Conv1D)             (None, None, 8)           904       
                                                                 
 global_average_pooling1d (G  (None, 8)                0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout (Dropout)           (None, 8)                 0         
                                                                 
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                        

In [15]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)
     

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.TensorBoard(log_dir="logs")
    ]
)

Epoch 1/10
625/625 [==============================] - 19s 27ms/step - loss: 0.6930 - accuracy: 0.5019 - val_loss: 0.6912 - val_accuracy: 0.4924
Epoch 2/10
625/625 [==============================] - 4s 6ms/step - loss: 0.6891 - accuracy: 0.5034 - val_loss: 0.6847 - val_accuracy: 0.4946
Epoch 3/10
625/625 [==============================] - 4s 6ms/step - loss: 0.6819 - accuracy: 0.5173 - val_loss: 0.6767 - val_accuracy: 0.5070
Epoch 4/10
625/625 [==============================] - 4s 6ms/step - loss: 0.6801 - accuracy: 0.5218 - val_loss: 0.6755 - val_accuracy: 0.5182
Epoch 5/10
625/625 [==============================] - 4s 6ms/step - loss: 0.6767 - accuracy: 0.5268 - val_loss: 0.6723 - val_accuracy: 0.5352
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.6744 - accuracy: 0.5349 - val_loss: 0.6716 - val_accuracy: 0.5330
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.6734 - accuracy: 0.5351 - val_loss: 0.6696 - val_accuracy: 0.5372
Epoc

In [17]:
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 2212), started 7 days, 0:07:12 ago. (Use '!kill 2212' to kill it.)

In [18]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

782/782 [==============================] - 114s 143ms/step - loss: 0.6685 - accuracy: 0.5326
Loss:  0.6684603095054626
Accuracy:  0.5325599908828735


In [19]:
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)


loss, accuracy = export_model.evaluate(raw_test_ds)
print(accuracy)

782/782 [==============================] - 13s 17ms/step - loss: 0.6685 - accuracy: 0.5914
0.5913599729537964


In [20]:
export_model.save("sentence_classification_model")

INFO:tensorflow:Assets written to: sentence_classification_model\assets


INFO:tensorflow:Assets written to: sentence_classification_model\assets
